In [1]:
!pip install selenium

     |████████████████████████████████| 911kB 2.7MB/s 


In [2]:
!pip install fake_useragent

  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-cp36-none-any.whl size=13484 sha256=f1190f0dcdd67165b4ad13c36faf3119d52761e33ce88988b316cb49ae214835
  Stored in directory: /root/.cache/pip/wheels/5e/63/09/d1dc15179f175357d3f5c00cbffbac37f9e8690d80545143ff
Successfully built fake-useragent


In [60]:
!pip install lxml

In [37]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [54]:
with open('/content/gdrive/My Drive/colab_data/test.txt', 'w') as f:
    f.write('hello')

!cat /content/gdrive/My\ Drive/colab_data/test.txt

# !cat /content/gdrive/My\ Drive/chromedriver.exe

hello

In [58]:
# from pyvirtualdisplay import Display # 도커 사용자가 필요함, 가상의 화면을 흉내냄
from selenium import webdriver #자주씀(필수)
from selenium.webdriver.common.keys import Keys #입력창에 글자 입력시 사용
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities #자주씀(필수) 나머지들은 옵션
from selenium.webdriver.support import expected_conditions as EC

from selenium.webdriver.support.ui import WebDriverWait 

from selenium.common.exceptions import TimeoutException #더보기 해줄때
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import time

options = webdriver.ChromeOptions()
options.add_argument('headless') #백그라운드에서 크롬창 열리게 하는 것_크롤링 용 / 크롤링 과정 눈으로 지켜보고 싶다면 끌것
options.add_argument('window-size=1920x1080')
options.add_argument('disable-gpu')#gpu 쓰지 않게 함
ua = {'User-Agent': UserAgent().chrome}
options.add_argument(ua)
options.add_argument('lang=ko_KR') #한국사이트일때 필요함

드라이버 로드


In [67]:
# remote_url = 'http://localhost:4444/wd/hub' #remote에서 셀레늄 부르는 #or backdriver
driver = webdriver.Chrome('./chromedriver.exe') #로컬 환경용
# driver = webdriver.Chrome('')
# driver = webdriver.Chrome() #도커에서 로컬환경
# driver = webdriver.Remote(remote_url, DesiredCapabilities.CHROME) #리모트 환경

# driver.quit() #셀레늄_드라이버 여러개 올리면 충돌남, 꼭 닫아줘야함

WebDriverException: ignored

In [ ]:
driver.get('https://news.v.daum.net/v/20200730050135622')
print(driver.title)
print(driver.current_url)
print(driver.find_element_by_id('harmonyContainer').text)
print(driver.find_element_by_xpath('//*[@id="harmonyContainer"]').text)

In [ ]:
bs = BeautifulSoup(driver.page_source, 'lxml')
print(bs.select('#harmonyContainer')[0].text)

In [ ]:
from selenium.webdriver.common.by import By
driver.get('https://news.daum.net/digital/')
link_viewall = WebDrierWait(driver, 2).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, '#myArticle > div > a'))
    ) #객체 들고 오기

webdriver.ActionChains(driver).click(link_viewall).perform() #클릭 호출

html = driver.page_source
bs = BeautifulSoup(html, 'lxml')
# bs.text
# print(bs.select('#myArticle > div.box_etc > ul > li:nth-child(1) > div > strong > a')[0])
# print(bs.select('strong.tit_thumb')[0].text)

In [ ]:
driver.get('http://pythonscraping.com/pages/files/form.html')
firstnameField = driver.find_element_by_name('firstname')
lastnameField = driver.find_element_by_name('lastname')
submitButton = driver.find_element_by_id('submit')

In [ ]:
actions = webdriver.ActionChains(driver).click(firstnameField).send_keys('JungHyun') \ 
        .click(lastnameField).send_keys('Son').send_keys(Keys.RETURN)

actions.perform()

print(driver.find_element_by_tag_name('body').text)


In [ ]:
driver.get('https://news.v.daum.net/v/20200730105256713')
loop, count = True, 0
while loop and count < 10:
    try:
        element = WebDriverWait(driver, 5).until(
              EC.presence_of_all_elements_located(By.CSS_SELECTOR, '#alex-area > div > div > div > ')
        )

        more_button = driver.find_element_by_css_selector('#alex-area > div > div > div.cmt_box >')
        webdriver.ActionChains(driver).click(more_button).perform()
        count += 1
        time.sleep(1)
        print(count)
    except TimeoutException:
        loop = False

comment_ul = driver.find_element_by_css_selector('#alex-area > div > div > div.cmt_box >')
commnet_li = comment_ul.find_elements_by_tag_name('li')
for num, comment_item in enumerate(comment_li):
    print('[' + str(num) + ']', comment_item.find_element_by_css_selector('div >  p').text